In [ ]:
import numpy as np
import pandas as pd

import nltk


In [ ]:
!git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

Cloning into 'DrugReviewAnalysis'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 51 (delta 5), reused 3 (delta 0), pack-reused 37
Receiving objects: 100% (51/51), 72.00 MiB | 6.33 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
# df_train = pd.read_csv("DrugReviewAnalysis/drugsComTrain_raw.tsv", sep="\t")
# df_test = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")

df_train = pd.read_csv("DrugReviewAnalysis/drugsComTrain_raw.tsv", sep="\t")
df_test = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")

In [ ]:
df_train = df_train[df_train["condition"].str.contains("users found this comment helpful.")==False]
df_test = df_test[df_test["condition"].str.contains("users found this comment helpful.")==False]

In [ ]:
len(set(df_train["condition"]) & set(df_test["condition"])), len(set(df_test["condition"]))

(639, 664)

In [ ]:
len(set(df_train["drugName"]) & set(df_test["drugName"])), len(set(df_test["drugName"]))

(2387, 2629)

In [ ]:
df_train["timestamp"] = df_train["date"].apply(lambda x: pd.Timestamp(x))
df_test["timestamp"] = df_test["date"].apply(lambda x: pd.Timestamp(x))

In [ ]:
df_train = df_train.drop(columns="Unnamed: 0")

In [ ]:
df_test = df_test.drop(columns="Unnamed: 0")

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
def preprocess(text):
    tokens = word_tokenize(text.lower())

    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = " ".join(lemmatized_tokens)
    return processed_text


In [13]:
#df_train["condition"] = df_train["condition"].head(5).apply(lambda x: spell(x))

In [14]:
df_train["condition"] = df_train["condition"].apply(lambda x: str(x).lower())

In [15]:
np.sort(df_train["condition"].unique())

array(['abdominal distension', 'abnormal uterine bleeding', 'abortion',
       'acetaminophen overdose', 'acial lipoatrophy', 'acial wrinkles',
       'acne', 'actinic keratosis', 'actor ix deficiency',
       'acute coronary syndrome', 'acute lymphoblastic leukemia',
       'acute nonlymphocytic leukemia', 'acute promyelocytic leukemia',
       "addison's disease", 'adhd', 'adrenocortical insufficiency',
       'adult human growth hormone deficiency', 'aggressive behavi',
       'agitated state', 'agitation', 'aids related wasting',
       'ailure to thrive', 'alcohol dependence', 'alcohol withdrawal',
       'allergic reactions', 'allergic rhinitis', 'allergic urticaria',
       'allergies', 'alopecia', 'alpha-1 proteinase inhibitor deficiency',
       "alzheimer's disease", 'amebiasis', 'amenorrhea',
       'amilial cold autoinflammatory syndrome',
       'amilial mediterranean feve', 'amyotrophic lateral sclerosis',
       'anal fissure and fistula', 'anal itching', 'anaphylaxis',


In [16]:
# df_train.to_csv("DrugReviewAnalysis/drugsTrain_processed.csv", sep="\t", index=False)

df_train.to_csv("drugsTrain_processed.csv", sep="\t", index=False)

**So far only ran for the `condition` column of the training dataset. Took more than an hour to spellcheck the column with the current code, but using the nltk library runs much faster. Need to figure out how to speed up spell checking to use on the `review` feature of the dataset.**

In [17]:
df_train[df_train["condition"].str.contains("addison")]

,drugName,condition,review,rating,date,usefulCount,timestamp
1195,Decadron,addison's disease,"""Longer acting steroid option.\r\nGained weigh...",8.0,"August 6, 2017",0,2017-08-06
21819,Dexamethasone,addison's disease,"""Smooth feel and effective. Start slow 0.5 mg ...",9.0,"November 13, 2012",9,2012-11-13
23097,Dexamethasone,addison's disease,"""Taking this medication 0.25 mg daily with 0.0...",10.0,"April 5, 2013",11,2013-04-05
25357,Hydrocortisone,addison's disease,"""This medication is life sustaining. I take a...",10.0,"October 3, 2017",0,2017-10-03
48790,Hydrocortisone,addison's disease,"""Added so much weight,due to this drugs""",8.0,"January 10, 2016",4,2016-01-10
67629,Dexamethasone,addison's disease,"""I had weight gain.""",9.0,"December 8, 2013",4,2013-12-08
84321,Hydrocortone,addison's disease,"""Added so much weight,due to this drugs""",8.0,"January 10, 2016",4,2016-01-10
110102,Hydrocortone,addison's disease,"""Feel better,but getting really fat""",8.0,"February 9, 2015",7,2015-02-09
118723,Hydrocortisone,addison's disease,"""Feel better,but getting really fat""",8.0,"February 9, 2015",7,2015-02-09
122902,Fludrocortisone,addison's disease,"""Works like a charm and I have zero side effec...",10.0,"July 29, 2009",37,2009-07-29


In [18]:
df_train["review"] = df_train["review"].str.replace("&#039;", "'")

In [19]:
np.sort(df_train["condition"].unique())

array(['abdominal distension', 'abnormal uterine bleeding', 'abortion',
       'acetaminophen overdose', 'acial lipoatrophy', 'acial wrinkles',
       'acne', 'actinic keratosis', 'actor ix deficiency',
       'acute coronary syndrome', 'acute lymphoblastic leukemia',
       'acute nonlymphocytic leukemia', 'acute promyelocytic leukemia',
       "addison's disease", 'adhd', 'adrenocortical insufficiency',
       'adult human growth hormone deficiency', 'aggressive behavi',
       'agitated state', 'agitation', 'aids related wasting',
       'ailure to thrive', 'alcohol dependence', 'alcohol withdrawal',
       'allergic reactions', 'allergic rhinitis', 'allergic urticaria',
       'allergies', 'alopecia', 'alpha-1 proteinase inhibitor deficiency',
       "alzheimer's disease", 'amebiasis', 'amenorrhea',
       'amilial cold autoinflammatory syndrome',
       'amilial mediterranean feve', 'amyotrophic lateral sclerosis',
       'anal fissure and fistula', 'anal itching', 'anaphylaxis',


In [20]:
df_train[df_train["condition"] == "mist ("]

,drugName,condition,review,rating,date,usefulCount,timestamp
107305,Formoterol,mist (,"""I have used this for about 2 years and it sur...",9.0,"August 20, 2011",23,2011-08-20


In [21]:
import re

In [22]:
# pip install scikit-learn

In [23]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

In [24]:
dn_train_ohe = ohe.fit_transform(df_train["drugName"].values.reshape(-1,1)).toarray()
cond_train_ohe = ohe.fit_transform(df_train["condition"].values.reshape(-1,1)).toarray()

In [25]:
dn_train_ohe.shape, cond_train_ohe.shape

((159498, 3412), (159498, 811))

In [26]:
dn_test_ohe = ohe.fit_transform(df_test["drugName"].values.reshape(-1,1)).toarray()
cond_test_ohe = ohe.fit_transform(df_test["condition"].values.reshape(-1,1)).toarray()

In [27]:
dn_test_ohe.shape, cond_test_ohe.shape

((53200, 2629), (53200, 664))

In [28]:
df_train["pair"] = list(df_train["drugName"] + " " + df_train["condition"])


In [29]:
df_train.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,pair
0,Valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20,Valsartan left ventricular dysfunction
1,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27,Guanfacine adhd
2,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14,Lybrel birth control
3,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03,Ortho Evra birth control
4,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27,Buprenorphine / naloxone opiate dependence


In [30]:
pairs = ohe.fit_transform(df_train["pair"].values.reshape(-1,1)).toarray()
pairs.shape

(159498, 7819)

In [31]:
#lowercase the train data
df_train['processed_review'] = df_train['review'].apply(lambda x: str(x).lower())

In [32]:
#lowercase the test data
df_test['processed_review'] = df_test['review'].apply(lambda x: str(x).lower())

In [33]:
df_train.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,pair,processed_review
0,Valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20,Valsartan left ventricular dysfunction,"""it has no side effect, i take it in combinati..."
1,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27,Guanfacine adhd,"""my son is halfway through his fourth week of ..."
2,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14,Lybrel birth control,"""i used to take another oral contraceptive, wh..."
3,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03,Ortho Evra birth control,"""this is my first time using any form of birth..."
4,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27,Buprenorphine / naloxone opiate dependence,"""suboxone has completely turned my life around..."


In [34]:
df_test.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,"""i&#039;ve tried a few antidepressants over th..."
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,"""my son has crohn&#039;s disease and has done ..."
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,"""quick reduction of symptoms"""
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,"""contrave combines drugs that were used for al..."
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,"""i have been on this birth control for one cyc..."


In [35]:
df_train['processed_review'] = df_train['processed_review'].str.replace('&#039;', "'", regex=False)
df_test['processed_review'] = df_test['processed_review'].str.replace('&#039;', "'", regex=False)

In [36]:
df_test.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,"""i've tried a few antidepressants over the yea..."
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,"""my son has crohn's disease and has done very ..."
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,"""quick reduction of symptoms"""
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,"""contrave combines drugs that were used for al..."
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,"""i have been on this birth control for one cyc..."


In [37]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install nltk
!pip install punkt
df_train.loc[:1000, "processed_review"] = df_train.loc[:1000,"processed_review"].apply(preprocess)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [38]:
df_train[:1000]

,drugName,condition,review,rating,date,usefulCount,timestamp,pair,processed_review
0,Valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20,Valsartan left ventricular dysfunction,"`` side effect , take combination bystolic 5 m..."
1,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27,Guanfacine adhd,`` son halfway fourth week intuniv . became co...
2,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14,Lybrel birth control,"`` used take another oral contraceptive , 21 p..."
3,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03,Ortho Evra birth control,`` first time using form birth control . 'm gl...
4,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27,Buprenorphine / naloxone opiate dependence,`` suboxone completely turned life around . fe...
...,...,...,...,...,...,...,...,...,...
1011,Ranitidine,gerd,"""Zantac worked wonders on me. I have taken thi...",10.0,"May 12, 2012",44,2012-05-12,Ranitidine gerd,"""zantac worked wonders on me. i have taken thi..."
1012,Citalopram,depression,"""Works great. I feel like myself again- positi...",8.0,"April 18, 2014",79,2014-04-18,Citalopram depression,"""works great. i feel like myself again- positi..."
1013,Arava,rheumatoid arthritis,"""I I could not take it stomach problems and ba...",1.0,"November 20, 2015",31,2015-11-20,Arava rheumatoid arthritis,"""i i could not take it stomach problems and ba..."
1015,Implanon,birth control,"""I got the Implanon 3 years ago and have had n...",10.0,"May 8, 2014",3,2014-05-08,Implanon birth control,"""i got the implanon 3 years ago and have had n..."


In [40]:
df_test.loc[:1000, "processed_review"] = df_test.loc[:1000,"processed_review"].apply(preprocess)
df_test[:1000]

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,`` 've tried antidepressant year ( citalopram ...
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,`` son crohn 's disease done well asacol . com...
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,`` quick reduction symptom ``
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,"`` contrave combine drug used alcohol , smokin..."
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,`` birth control one cycle . reading review ty...
...,...,...,...,...,...,...,...,...
1006,Polyethylene glycol 3350 with electrolytes,Bowel Preparation,"""The best way to drink this is put it in a ble...",8.0,"March 3, 2013",11,2013-03-03,"""the best way to drink this is put it in a ble..."
1007,Bupropion,Depression,"""I had constant diarrhea, 3-4 times a day. I h...",1.0,"August 28, 2015",11,2015-08-28,"""i had constant diarrhea, 3-4 times a day. i h..."
1008,Paxil,Generalized Anxiety Disorde,"""Taking 20mg. Thought it was starting to help ...",5.0,"March 5, 2009",21,2009-03-05,"""taking 20mg. thought it was starting to help ..."
1009,Ethinyl estradiol / norethindrone,Birth Control,"""Rating it as a 3 only because it kept me from...",3.0,"February 28, 2017",7,2017-02-28,"""rating it as a 3 only because it kept me from..."
